In [35]:
from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
from selenium.webdriver.common.keys import Keys
service = Service(executable_path=r"C:\Users\ZACK LEE\Documents\Coding\chromedriver_win32\chromedriver.exe")
op = webdriver.ChromeOptions()
op.add_argument('headless')

In [24]:
LOGIN_DETAILS_FILE_PATH = r"C:\Users\ZACK LEE\Documents\Username&Password.txt"
websites = { 
                'jobstreet': {
                    
                                'url' : "https://myjobstreet.jobstreet.com.my/home/login.php?site=my&language_code=3",
                                'login_id' : "login_id",
                                'password_id' : "password",
                                'btn_id' : "btn_login",
                                'search_content' : "https://www.jobstreet.com.my/en/job-search/data-science-jobs/",
                                'job_lists_path' : '//*[@id="jobList"]/div[2]/div[2]/div',
                                'job_path' : 'div',
                                'job_url_path' : 'div/div/article/div/div/div[1]/div[1]//a[@href]',
                                'role_path' : 'div/div/article/div/div/div[1]/div[1]//h1/a/div/span',
                                'company_path' : 'div/div/article/div/div/div[1]/div[1]/div[3]/span',
                                'location_path' : 'div/div/article/div/div/div[1]/div[1]/span/span',
                                'description_path' : '//*[@id="contentContainer"]/div/div[2]/div/div[1]/div',
                                'next_page_first' : '//*[@id="jobList"]/div[2]/div[3]/div/a',
                                'next_page' : '//*[@id="jobList"]/div[2]/div[3]/div/a[2]'
                },
                
                'linkedin' : {
                    
                                'url' : "https://www.linkedin.com/checkpoint/rm/sign-in-another-account?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin",
                                'login_id' : "username",
                                'password_id' : "password",
                                'btn_id' : "login__form_action_container",
                                'search_content' : "https://www.linkedin.com/jobs/search/?keywords=data%20science",
                                'job_lists_path' : '/html/body/div[6]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul',
                                'job_path' : 'li',
                                'job_url_path' : 'div/div/div[1]/div[2]/div[1]/a',
                                'role_path' : 'div/div/div[1]/div[2]/div[1]/a',
                                'company_path' : 'div/div/div[1]/div[2]/div[2]/a',
                                'location_path' : 'div/div/div[1]/div[2]/div[3]/ul/li',
                                'description_path' : '//*[@id="job-details"]/span'
                    
                }
}
#ember186

In [181]:
def get_login_details(path):
    with open(path, "r") as file1:
    # Reading form a file
        username = file1.readline()
        password = file1.readline()
    return username,password

def login(driver,un,p,webname):
    url = websites[webname]['url']
    wait = WebDriverWait(driver, 10)
    driver.get(url)
    username = driver.find_element(By.ID,websites[webname]['login_id'])
    username.send_keys(un)
    password = driver.find_element(By.ID,websites[webname]['password_id'])
    password.send_keys(p)
    
    ## for jobstreet
    # driver.find_element(By.ID,websites[webname]['btn_id']).click()
    
    # for linkedin
    driver.find_element(By.CLASS_NAME,websites[webname]['btn_id']).click()
    
def search(driver,webname):
    time.sleep(5)
    driver.get(websites[webname]['search_content'])

def get_jobs(driver,job_details,webname):

    try:
        job_lists = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH, websites[webname]['job_lists_path'])))
    except:
        driver.get(driver.current_url);
        job_lists = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH, websites[webname]['job_lists_path'])))
    # job_lists = driver.find_element(By.XPATH,'//*[@id="jobList"]/div[2]/div[2]/div')
    jobs = job_lists.find_elements(By.XPATH,websites[webname]['job_path'])
    # print(len(jobs))
    for job in jobs:
        ## skip the current data if error occurs while collecting it
        try:
            url = job.find_element(By.XPATH,websites[webname]['job_url_path']).get_attribute("href")
            role = job.find_element(By.XPATH,websites[webname]['role_path']).text
            company = job.find_element(By.XPATH,websites[webname]['company_path']).text
            location = job.find_element(By.XPATH,websites[webname]['location_path']).text
            # print(role)
        except:
            continue
        job_details.update({url:{'company':company,'location':location,'role':role}})

        
def get_description(driver,job_dict,webname):
    
    for url in list(job_dict.keys()):
        time.sleep(1)
        try:
            driver.get(url)
            driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div/div[1]/div[1]/div/div[2]/footer/button').click()
            description = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH, websites[webname]['description_path']))).text
            
        except:
            driver.get(url)
            driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div/div[1]/div[1]/div/div[2]/footer/button').click()
            description = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,  websites[webname]['description_path']))).text
        print(description)
        
        job_dict.get(url).update({"description":description})
        
    
def load_next_page(driver,first,webname):
    
    if(first):
        next = driver.find_element(By.XPATH, websites[webname]['next_page_first'])
        next.click()
    else:
        next = driver.find_element(By.XPATH, websites[webname]['next_page'])
        next.click()
        
def scroll_down(driver):
    
    try:
        job_lists = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH, websites[webname]['job_lists_path'])))
    except:
        driver.get(driver.current_url);
        job_lists = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH, websites[webname]['job_lists_path'])))
    # job_lists = driver.find_element(By.XPATH,websites[webname]['job_lists_path'])
    jobs = job_lists.find_elements(By.XPATH,websites[webname]['job_path'])
    # print(len(jobs))
    for job in jobs:
        name = job.get_attribute('id');
        driver.execute_script(f'document.querySelector("#{name} > div > div").scrollIntoView();')
        time.sleep(1)
        
def load_next_page_linkedin(driver):
  #loads next page for url retrival
    curr= driver.find_element(By.XPATH,'//*[@aria-current="true"]').text
    next = driver.find_element(By.XPATH,f'//*[@aria-label="Page {int(curr)+1}"]')
    next.click()

In [ ]:

webname = 'linkedin'
driver = webdriver.Chrome(service=service)
username,password = get_login_details(LOGIN_DETAILS_FILE_PATH)
job_dict = {}
login(driver,username,password,webname)
search(driver,webname)
isfirst = True
i = 0
while True:
    i+=1
    print(i)
    print(len(list(job_dict)))
    try:
        scroll_down(driver)
        get_jobs(driver,job_dict,webname)
        # load_next_page(driver,isfirst,webname)
        load_next_page_linkedin(driver)
    except:

        break
    isfirst = False
# get_description(driver,job_dict,webname)



In [186]:
import pickle

def save(final_file):
    with open(f'./data/job_new_dict_linkedin.p', 'wb') as fp:
        pickle.dump(final_file,file=fp,protocol=pickle.HIGHEST_PROTOCOL)
def load():
    with open('./data/job_dict.p', 'rb') as fp:
        job_dict = pickle.load(fp)

In [187]:
save(job_dict)

In [175]:
webname = 'linkedin'
driver = webdriver.Chrome(service=service)
username,password = get_login_details(LOGIN_DETAILS_FILE_PATH)
login(driver,username,password,webname)

In [ ]:
get_description(driver,job_dict,webname)
# driver.get(list(job_dict.keys())[0])